<a href="https://colab.research.google.com/github/Elwing-Chou/ml0930/blob/master/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
f = zipfile.ZipFile("drive/My Drive/additional/train.zip")
f.extractall()

In [ ]:
import pandas as pd
import glob
dogs = glob.glob("train/dog.*")
cats = glob.glob("train/cat.*")
df = pd.DataFrame({
    "path":dogs + cats,
    "target":[0] * len(dogs) + [1] * len(cats)
})
df

In [ ]:
import random
from tensorflow.keras.preprocessing.image import load_img
idx = random.randint(0, 24999)
load_img(df["path"][idx], target_size=(224, 224))

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
x, y = np.array(df["path"]), np.array(df["target"])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
cnn = vgg16.VGG16(include_top=False, input_shape=(224, 224, 3))
# freeze一定要在Compile之前
for l in cnn.layers:
    l.trainable = False
mlp = [
    Flatten(),
    Dense(128, activation="relu"),
    Dense(2, activation="softmax")
]
model = Sequential(cnn.layers + mlp)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

In [ ]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
import numpy as np

def getdata(x, y, batch=20):
    idx = np.random.randint(0, x.shape[0], size=batch)
    orix, batchx, batchy = [], [], y[idx]
    for p in x[idx]:
        img = load_img(p, target_size=(224, 224))
        img_np = np.array(img)
        orix.append(img_np)
        img_pre = vgg16.preprocess_input(img_np)
        batchx.append(img_pre)
    return (np.array(orix), np.array(batchx), np.array(batchy))

testori, testx, testy = getdata(x_train, y_train, 20)
print(testori.shape)
print(testx.shape)
print(testy.shape)